In [1]:
# Install the Antigranular package
!pip install antigranular &> /dev/null

In [2]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Fraud Detection Hackathon with Privacy Village #DPD25FEST")

Dataset "Transaction Fraud Hackathon Dataset" loaded to the kernel as transaction_fraud_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
test_x                         DataFrame      
train_x                        PrivateDataFrame
train_y                        PrivateDataFrame

Connected to Antigranular server session id: 72c711ea-4aac-4bbc-93bb-decf3a104868, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [3]:
%%ag
import tensorflow as tf
from op_pandas import standard_scaler, PrivateDataFrame
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from op_tensorflow import PrivateKerasModel, PrivateDataLoader
from op_diffprivlib.models import GaussianNB
import pandas as pd


/usr/local/lib/python3.10/site-packages/tensorflow/__init__.py:30: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils as _distutils
2025-03-02 20:54:45.734319: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.

2025-03-02 20:54:45.737470: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-02 20:54:45.933929: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-02 20:54:45.933955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unabl

In [4]:
%%ag
x_train = transaction_fraud_hackathon_dataset["train_x"]
y_train = transaction_fraud_hackathon_dataset["train_y"]
x_test = transaction_fraud_hackathon_dataset["test_x"]

In [5]:
%%ag
x_train_scaled = standard_scaler(x_train, eps=0.5)  # Lower epsilon for privacy
x_test_scaled = standard_scaler(PrivateDataFrame(x_test), eps=0.1)

In [6]:
%%ag
gnb = GaussianNB(epsilon=0.3, bounds=([0]*9, [1]*9))
gnb.fit(x_train_scaled, y_train)
y_pred_gnb = gnb.predict(x_test_scaled)

/usr/local/lib/python3.10/site-packages/ag_wrapper/wrapper.py:129: UserWarning: Bounds estimation epsilon is set to 0.1. It wont be charged until a private dataframe / series ispassed to predict or transform methods.
  warnings.warn(f"Bounds estimation epsilon is set to {bounds_estimation_eps}. It wont be charged until "
/usr/local/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



In [8]:
%%ag
seqM = Sequential([
    Dense(64, activation='relu', input_shape=(9,)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary Classification
])

dp_model = PrivateKerasModel(model=seqM, l2_norm_clip=1, noise_multiplier=0.5)  # Low noise for better performance
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

dp_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=["accuracy"])

In [ ]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train_scaled, label_df=y_train, batch_size=8)
dp_model.fit(x=data_loader, epochs=3, target_delta=0.0001)  # Balance privacy & accuracy

In [10]:
%%ag
y_pred_dp = dp_model.predict(x_test_scaled, label_columns=["output"])

469/469 [==============================] - 3s 6ms/step



In [11]:
%%ag
def threshold(x: float) -> float:
    return 1 if x > 0.5 else 0
y_pred_dp["output"] = y_pred_dp["output"].map(threshold, output_bounds=(0, 1))

In [ ]:
%%ag
final_pred = (0.6 * y_pred_gnb["output"]) + (0.4 * y_pred_dp["output"])
final_pred = final_pred.map(lambda x: 1 if x > 0.5 else 0)

In [ ]:
%%ag
from pandas import DataFrame
result = submit_predictions(DataFrame(final_pred))
print(result)